<hr>

# ST1511 AI and Machine Learning CA1 Assignment (Regression)
<hr>

Name: Lee Hong Yi <br>
Class: DAAA/FT/2A/05 <br>
Admin Number: p2223010 <br>

__Objective:__ <br>
Predict the price of a house in the United States based on various factors.

__Background Information:__<br>
The price of a house is based on several factors, such as it's location, area, and amenities, such as bedrooms, toilets, and renovation status.


__Additional Information:__ <br>

<hr>
<font size = 5>Importing Libraries</font>
<hr>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, make_scorer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import ShuffleSplit, GridSearchCV, RandomizedSearchCV, train_test_split

from warnings import simplefilter


simplefilter(action='ignore', category=FutureWarning)

<font size = 5> Preparing data.</font> <br>

Since the data has already been collected, we can move straight onto preparing the data for training by the model.

In [ ]:
original_data = pd.read_csv("./housing_dataset.csv")
eda_data = original_data.copy()


In [ ]:
eda_data.describe()

In [ ]:
display(eda_data.value_counts("City"))
display(eda_data.value_counts("No. of Bedrooms"))
display(eda_data.value_counts("No. of Toilets"))
display(eda_data.value_counts("Stories"))

In [ ]:
fig, ax = plt.subplots(1,5, figsize = (25,10))

sns.boxplot(data = eda_data, ax = ax[0], y = "Price ($)")
sns.boxplot(data = eda_data, ax = ax[1], y = "House Area (sqm)")
sns.boxplot(data = eda_data, ax = ax[2], y = "No. of Bedrooms")
sns.boxplot(data = eda_data, ax = ax[3], y = "No. of Toilets")
sns.boxplot(data = eda_data, ax = ax[4], y = "Stories")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (30,10))
fig = sns.kdeplot(data = eda_data[eda_data["City"] == "New York"]["Price ($)"], color = "black", label = "New York")
fig2 = sns.kdeplot(data = eda_data[eda_data["City"] == "Seattle"]["Price ($)"], color = "red", label = "Seattle")
fig3 = sns.kdeplot(data = eda_data[eda_data["City"] == "Boston"]["Price ($)"], color = "blue", label = "Boston")
fig4 = sns.kdeplot(data = eda_data[eda_data["City"] == "Denver"]["Price ($)"], color = "green", label = "Denver")
fig5 = sns.kdeplot(data = eda_data[eda_data["City"] == "Chicago"]["Price ($)"], color = "orange", label = "Chicago")
plt.suptitle("Distribution of price based on city (Hundred Thousand USD)")
ax.legend()
plt.show()


From the above graph, we can see that the city of Denver has houses that are higher on average, as it's price distribution curve is skewed further to the right as compared to the other cities.

In [ ]:
fig, ax = plt.subplots(figsize = (30,10))
fig = sns.kdeplot(data = eda_data[eda_data["City"] == "New York"]["House Area (sqm)"], color = "black", label = "New York")
fig2 = sns.kdeplot(data = eda_data[eda_data["City"] == "Seattle"]["House Area (sqm)"], color = "red", label = "Seattle")
fig3 = sns.kdeplot(data = eda_data[eda_data["City"] == "Boston"]["House Area (sqm)"], color = "blue", label = "Boston")
fig4 = sns.kdeplot(data = eda_data[eda_data["City"] == "Denver"]["House Area (sqm)"], color = "green", label = "Denver")
fig5 = sns.kdeplot(data = eda_data[eda_data["City"] == "Chicago"]["House Area (sqm)"], color = "orange", label = "Chicago")
plt.suptitle("Distribution of house size in square meters")
ax.legend()
plt.show()

From the above graph, we can also see that the city of Denver has houses that are larger on average, as it's distribution curve is skewed further to the right, while also having a lower proportion below ~400sqm as compared to other cities. In addition, the city of New York has more houses that are around 300 sqm, given that the graph has a higher peak at that point on the scale. 

Creating a correlogram to highlight the most correlated variables in the data table.

In [ ]:
plt.figure(figsize = (16,6))
sns.heatmap(eda_data.corr(), annot = True, linewidths= 0.5)
plt.show()

<font size = 5>Feature Engineering</font>

Using the "House Area (sqm)" column, as well as the "Price ($)" column to calculate a new feature, called "Price / PSM".

In [ ]:
for row in analysis_data.itertuples():
    print(row)
    print(row[1])
# analysis_data["psm_price"] = analysis_data["House Area (sqm)"] / analysis_data["Price($)"]

<font size = 5>Preparing the data for training.</font>

In [ ]:

target_data = pd.DataFrame(original_data["Price ($)"])
analysis_data = original_data.drop(columns = ["House ID",'Price ($)'])

display(analysis_data.head())
display(target_data.head())

analysis_data = pd.get_dummies(analysis_data)
display(analysis_data.head())
X_train, X_test, y_train, y_test = train_test_split(analysis_data, target_data, test_size=0.3, random_state=42)

<font size = 5>Choosing / training a model</font>

In [ ]:
steps = [("scaler", StandardScaler()),
         ("", Lasso(alpha=0.5))]

<font size = 5 >Evaluating the model</font>